## knock 077 統計的に外れ値を除外する

In [2]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列の表示が省略されないようにする
import polars.selectors as cs# 抽出条件のプリセット

### データを読み込む

In [3]:
df_receipt = pl.read_csv("../docker/work/data/receipt.csv")
display(df_receipt.head())

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001…","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097…","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014…","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000…","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050…","""P060102007""",1,90


### ノック

In [8]:
(
    df_receipt
    # 使用する列を抽出する(大規模データの場合メモリ使用量を節約する効果がある)
    .select([
        pl.col("customer_id"),
        pl.col("amount")
    ])
    # 顧客別に層別する
    .group_by(by = "customer_id")
    # 売上高を計算する
    .agg([
        pl.sum("amount").alias("sum_amount")
    ])
    # 売上高の常用対数を計算する
    .with_columns(
        pl.col("sum_amount").log10().alias("log_sum_amount")
    )
    # 売上高が平均から±3σを超えて離れた顧客データを抽出する
    .filter(
        (pl.col("log_sum_amount") > pl.mean("log_sum_amount") + 3 * pl.std("log_sum_amount", ddof = 0))
        |
        (pl.col("log_sum_amount") < pl.mean("log_sum_amount") - 3 * pl.std("log_sum_amount", ddof = 0))
    )
)

customer_id,sum_amount,log_sum_amount
str,i64,f64
"""ZZ000000000000…",12395003,7.093247
